In [1]:
from google.colab import drive
import sys
drive.mount('/content/drive')

FOLDERNAME = 'test_ml/'
assert FOLDERNAME is not None, "[!] Enter the foldername."

sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

Mounted at /content/drive


##Подключение к устройству



In [2]:
import torch

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print('using device:', device)

using device: cpu


##Датасеты
Загрузка

In [3]:
import my_dataset

path_from = "drive/MyDrive/test_ml/dataset/train.zip"
path_to = "drive/MyDrive/test_ml/dataset/"

loaded_data = my_dataset.DataLoad(path_from, path_to)
loaded_data.data.head()

,filename,label
0,cat.0.jpg,cat
1,cat.1.jpg,cat
2,cat.10.jpg,cat
3,cat.100.jpg,cat
4,cat.1000.jpg,cat


Разбиение датасета на 3 выборки: train = 80%, validation = 10%, test = 10%

In [4]:
x_train, x_valid, x_test = loaded_data.split_data()

print('Размер тренировочного набора данных', x_train.shape)
print('Размер валидационного набора данных', x_valid.shape)
print('Размер тестового набора данных', x_test.shape)

Размер тренировочного набора данных (20000, 2)
Размер валидационного набора данных (2500, 2)
Размер тестового набора данных (2500, 2)


Аугментация

In [5]:
from my_dataset import CustomDataset

train_dataset = CustomDataset(data=x_train, img_dir=path_to + "train/", transform=my_dataset.train_augmentation)
valid_dataset = CustomDataset(data=x_valid, img_dir=path_to + "train/", transform=my_dataset.test_augmentation)
test_dataset = CustomDataset(data=x_test, img_dir=path_to + "train/", transform=my_dataset.test_augmentation)

Разметка на батчи

In [6]:
from torch.utils.data import DataLoader

batch_size = 64 # размер батча
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

##Тренировка модели
Модель представляет из себя сверточную нейросеть.

In [7]:
import model
from train import train_model, test_model

net = model.Model().to(device)

loss_history, y_pred_train = train_model(train_dataset, net, epochs=25, device=device)


{'image': array([[[203, 164,  87],
        [203, 164,  87],
        [204, 165,  88],
        ...,
        [240, 201, 122],
        [239, 200, 121],
        [238, 199, 120]],

       [[203, 164,  87],
        [203, 164,  87],
        [204, 165,  88],
        ...,
        [241, 202, 123],
        [240, 201, 122],
        [238, 199, 120]],

       [[203, 164,  87],
        [203, 164,  87],
        [204, 165,  88],
        ...,
        [241, 202, 123],
        [240, 201, 122],
        [239, 200, 121]],

       ...,

       [[153, 122,  55],
        [153, 122,  55],
        [153, 122,  55],
        ...,
        [  2,   2,   0],
        [  2,   2,   0],
        [  2,   2,   0]],

       [[152, 121,  54],
        [152, 121,  54],
        [152, 121,  54],
        ...,
        [  2,   2,   0],
        [  2,   2,   0],
        [  2,   2,   0]],

       [[151, 120,  53],
        [151, 120,  53],
        [151, 120,  53],
        ...,
        [  1,   1,   0],
        [  1,   1,   0],
        [  1, 

AttributeError: ignored

Работа с validation набором

Работа с test набором

##Оценка качества и вывод графиков
Оценка качества будет производиться по метрике f1.

In [ ]:
import sklearn.metrics as metrics

y_true_train = x_train['label']
result = metrics.f1_score(y_true_train, y_pred_train)

print("Оценка качества F1:", result)


##Вывод графиков
Функция потерь

In [ ]:
from matplotlib import pyplot as plt

plt.figure(figsize=(20),dpi=200)

plt.title('График потерь',fontsize=15)
plt.xlabel('Итерации',fontsize=12)
plt.ylabel('Потери',fontsize=12)
plt.plot(loss_history)

plt.show()

Confusion matrix для validation набора

In [ ]:
from sklearn.metrics import confusion_matrix
import pandas as pd

conf_matrix_valid = confusion_matrix(y_valid, y_pred_valid)
conf_matrix_labels_valid = pd.DataFrame(conf_matrix_valid, columns = ['Positive', 'Negative'], index = ['True', 'False'])
conf_matrix_labels_valid

Сonfusion matrix для test набора

In [ ]:
conf_matrix_test = confusion_matrix(y_test, y_pred_test)
conf_matrix_labels_test = pd.DataFrame(conf_matrix_test, columns = ['Positive', 'Negative'], index = ['True', 'False'])
conf_matrix_labels_test